In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

2023-11-04 18:01:12.552022: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load and preprocess the CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
y_train_one_hot = to_categorical(y_train, 10)
y_test_one_hot = to_categorical(y_test, 10)

170498071/170498071 [==============================] - 132s 1us/step


In [3]:
# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base model
base_model.trainable = False


9406464/9406464 [==============================] - 1s 0us/step


In [4]:
# Add custom layers on top of the pre-trained base
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # We have 10 classes in CIFAR-10
])

# Compile the model
model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# Train the model
history = model.fit(x_train, y_train_one_hot,
                    batch_size=32,
                    epochs=10,
                    validation_data=(x_test, y_test_one_hot))

Epoch 1/10
1563/1563 [==============================] - 68s 41ms/step - loss: 1.9268 - accuracy: 0.3027 - val_loss: 1.8604 - val_accuracy: 0.3297
Epoch 2/10
1563/1563 [==============================] - 58s 37ms/step - loss: 1.8283 - accuracy: 0.3364 - val_loss: 1.8300 - val_accuracy: 0.3352
Epoch 3/10
1563/1563 [==============================] - 73s 47ms/step - loss: 1.7909 - accuracy: 0.3476 - val_loss: 1.8096 - val_accuracy: 0.3442
Epoch 4/10
1563/1563 [==============================] - 69s 44ms/step - loss: 1.7634 - accuracy: 0.3580 - val_loss: 1.8017 - val_accuracy: 0.3502
Epoch 5/10
1563/1563 [==============================] - 66s 42ms/step - loss: 1.7409 - accuracy: 0.3669 - val_loss: 1.7949 - val_accuracy: 0.3496
Epoch 6/10
1563/1563 [==============================] - 69s 44ms/step - loss: 1.7231 - accuracy: 0.3764 - val_loss: 1.7941 - val_accuracy: 0.3478
Epoch 7/10
1563/1563 [==============================] - 67s 43ms/step - loss: 1.7066 - accuracy: 0.3803 - val_loss: 1.7926 -

In [6]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test_one_hot)
print(f'Test accuracy: {accuracy:.3f}')

313/313 [==============================] - 10s 33ms/step - loss: 1.7983 - accuracy: 0.3531
Test accuracy: 0.353


In [7]:
# Fine-tune the entire model
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile the model after you make any changes to the
# `trainable` attribute of any layer, so that your changes are taken into account
model.compile(optimizer=Adam(lr=0.00001),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model again (this time fine-tuning the entire model)
history_fine = model.fit(x_train, y_train_one_hot,
                         batch_size=32,
                         epochs=10,
                         validation_data=(x_test, y_test_one_hot))



Epoch 1/10
1563/1563 [==============================] - 313s 191ms/step - loss: 2.1334 - accuracy: 0.3963 - val_loss: 2.0205 - val_accuracy: 0.2731
Epoch 2/10
1563/1563 [==============================] - 281s 179ms/step - loss: 1.3024 - accuracy: 0.5473 - val_loss: 2.0042 - val_accuracy: 0.3177
Epoch 3/10
1563/1563 [==============================] - 273s 175ms/step - loss: 1.2480 - accuracy: 0.5645 - val_loss: 1.6213 - val_accuracy: 0.4132
Epoch 4/10
1563/1563 [==============================] - 254s 162ms/step - loss: 1.1143 - accuracy: 0.6178 - val_loss: 1.4299 - val_accuracy: 0.6063
Epoch 5/10
1563/1563 [==============================] - 312s 200ms/step - loss: 1.1227 - accuracy: 0.6122 - val_loss: 2.2343 - val_accuracy: 0.3235
Epoch 6/10
1563/1563 [==============================] - 305s 195ms/step - loss: 1.4612 - accuracy: 0.4888 - val_loss: 2.6166 - val_accuracy: 0.1462
Epoch 7/10
1563/1563 [==============================] - 309s 198ms/step - loss: 1.2948 - accuracy: 0.5367 - val_

In [8]:
# Evaluate the model again
loss_fine, accuracy_fine = model.evaluate(x_test, y_test_one_hot)
print(f'Fine-tuned test accuracy: {accuracy_fine:.3f}')

313/313 [==============================] - 10s 30ms/step - loss: 2.1191 - accuracy: 0.2426
Fine-tuned test accuracy: 0.243
